# Preparation of concatenate data information file

In [1]:
import sys
import copy
from collections import defaultdict

import pandas as pd
import numpy as np

import sqlite3

In [2]:
# folder
root = "/workspace/tggate"

# seed for control sampling
seed=24771

# target organ
organ="Liver"

# path for data folders
ORIGINAL_DIR = f"{root}/data/original"
PROCESSED_DIR = f"{root}/data/processed"

In [3]:
ind_df = pd.read_csv(
    f"{ORIGINAL_DIR}/open_tggates_individual.csv", encoding="shift_jis"
)
image_df = pd.read_csv(
    f"{PROCESSED_DIR}/curated.csv"
)
path_df = pd.read_csv(
    f"{ORIGINAL_DIR}/open_tggates_pathology.csv", encoding="shift_jis"
)
compound_df = pd.read_csv(
    f"{ORIGINAL_DIR}/tggates_in_vivo_compound_info.txt", encoding="shift_jis", sep="\t"
)

display(ind_df.head())
display(image_df.head())
display(path_df.head())
display(compound_df.head())

C:\Users\Katsuhisa Morita\AppData\Local\Temp\ipykernel_1340\2404833871.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  ind_df = pd.read_csv(


,exp_id,group_id,individual_id,indv_id,compound_name,compound_abbr,compound_no,species,test_type,sin_rep_type,...,organ_wt_link,bw_link,fc_link,has_liver_pathology,liver_pathology_link,has_kidney_pathology,kidney_pathology_link,liver_pathological_image_link,kidney_pathological_image_link,id
0,40,1,1,40011,acetaminophen,APAP,1,Rat,in vivo,Single,...,40011.0,/list?exp_id=0040&group_id=01&individual_id=1,NaN,False,NaN,False,NaN,/list?exp_id=0040&group_id=01&individual_id=1&...,/list?exp_id=0040&group_id=01&individual_id=1&...,NaN
1,40,1,2,40012,acetaminophen,APAP,1,Rat,in vivo,Single,...,40012.0,/list?exp_id=0040&group_id=01&individual_id=2,NaN,False,NaN,False,NaN,/list?exp_id=0040&group_id=01&individual_id=2&...,/list?exp_id=0040&group_id=01&individual_id=2&...,NaN
2,40,1,3,40013,acetaminophen,APAP,1,Rat,in vivo,Single,...,40013.0,/list?exp_id=0040&group_id=01&individual_id=3,NaN,False,NaN,False,NaN,/list?exp_id=0040&group_id=01&individual_id=3&...,/list?exp_id=0040&group_id=01&individual_id=3&...,NaN
3,40,1,4,40014,acetaminophen,APAP,1,Rat,in vivo,Single,...,40014.0,/list?exp_id=0040&group_id=01&individual_id=4,NaN,False,NaN,False,NaN,/list?exp_id=0040&group_id=01&individual_id=4&...,/list?exp_id=0040&group_id=01&individual_id=4&...,NaN
4,40,1,5,40015,acetaminophen,APAP,1,Rat,in vivo,Single,...,40015.0,/list?exp_id=0040&group_id=01&individual_id=5,NaN,False,NaN,False,NaN,/list?exp_id=0040&group_id=01&individual_id=5&...,/list?exp_id=0040&group_id=01&individual_id=5&...,NaN


,EXP_ID,GROUP_ID,INDIVIDUAL_ID,COMPOUND_NAME,ORGAN,FILE_LOCATION,CAPTURE_NO,SPECIES,SINGLE_REPEAT_TYPE,ADMINISTRATION_ROUTE_TYPE,ANIMAL_AGE(week),SACRIFICE_PERIOD,DOSE,DOSE_UNIT
0,40,1,1,acetaminophen,Liver,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,1/1,Rat,Single,Gavage,6,3 hr,0,mg/kg
1,40,1,1,acetaminophen,Kidney,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,1/1,Rat,Single,Gavage,6,3 hr,0,mg/kg
2,40,1,2,acetaminophen,Liver,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,1/1,Rat,Single,Gavage,6,3 hr,0,mg/kg
3,40,1,2,acetaminophen,Kidney,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,1/1,Rat,Single,Gavage,6,3 hr,0,mg/kg
4,40,1,3,acetaminophen,Liver,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,1/1,Rat,Single,Gavage,6,3 hr,0,mg/kg


,sample_id,barcode,exp_id,group_id,individual_id,compound_name,dose_level,sacri_period,organ,find_type,topo_type,grade_type,sp_flg,body_weight_link,food_consumption_link,organ_link,id,pathological_image_link
0,0698011LR,No ChipData,698,1,1,1% cholesterol + 0.25% sodium cholate,Control,4 day,Liver,"Deposit, glycogen",Peripheral,slight,True,NaN,NaN,ORGA0010,NaN,/list?organ=Liver&exp_id=0698&group_id=01&indi...
1,0698025LR,No ChipData,698,2,5,1% cholesterol + 0.25% sodium cholate,Control,8 day,Liver,"Deposit, glycogen",Peripheral,slight,True,NaN,NaN,ORGA0010,NaN,/list?organ=Liver&exp_id=0698&group_id=02&indi...
2,0698051LR,003017906001,698,5,1,1% cholesterol + 0.25% sodium cholate,High,4 day,Liver,"Degeneration, fatty",Peripheral,slight,False,/list?exp_id=0698&group_id=05&individual_id=1,/list?exp_id=0698&group_id=05&individual_id=1,ORGA0010,NaN,/list?organ=Liver&exp_id=0698&group_id=05&indi...
3,0698052LR,No ChipData,698,5,2,1% cholesterol + 0.25% sodium cholate,High,4 day,Liver,"Degeneration, fatty",Peripheral,slight,False,NaN,NaN,ORGA0010,NaN,/list?organ=Liver&exp_id=0698&group_id=05&indi...
4,0698053LR,003017906002,698,5,3,1% cholesterol + 0.25% sodium cholate,High,4 day,Liver,"Degeneration, fatty",Peripheral,slight,False,/list?exp_id=0698&group_id=05&individual_id=3,/list?exp_id=0698&group_id=05&individual_id=3,ORGA0010,NaN,/list?organ=Liver&exp_id=0698&group_id=05&indi...


,COMPOUND_NAME,VEHICLE,Administration route,ORGAN,Single dose Low (mg/kg),Single dose Middle (mg/kg),Single dose High (mg/kg),Repeat dose Low (mg/kg),Repeat dose Middle (mg/kg),Repeat dose High (mg/kg)
0,acetaminophen,0.5% MC,Gavage,Liver,300,600,1000,300,600,1000
1,acetaminophen,0.5% MC,Gavage,Kidney,300,600,1000,300,600,1000
2,isoniazid,0.5% MC,Gavage,Liver,200,600,2000,50,100,200
3,carbon tetrachloride,corn oil,Gavage,Liver,30,100,300,30,100,300
4,phenobarbital,0.5% MC,Gavage,Liver,100,150,300,10,30,100


In [4]:
ind_df.columns = [i.upper() for i in ind_df.columns]
image_df.columns = [i.upper() for i in image_df.columns]
path_df.columns = [i.upper() for i in path_df.columns]
compound_df.columns = [i.upper() for i in compound_df.columns]

image_df = image_df[image_df["ORGAN"] == organ]
path_df = path_df[path_df["ORGAN"] == organ]
# sort kidney first and delete duplicates
compound_df = compound_df.sort_values(by="ORGAN") 
compound_df = compound_df.drop_duplicates(subset="COMPOUND_NAME")

ind_df["INDV_ID"] = (
    ind_df["EXP_ID"] * 1000 + ind_df["GROUP_ID"] *
    10 + ind_df["INDIVIDUAL_ID"]
)

image_df["INDV_ID"] = (
    image_df["EXP_ID"] * 1000 + image_df["GROUP_ID"] *
    10 + image_df["INDIVIDUAL_ID"]
)
path_df["INDV_ID"] = (
    path_df["EXP_ID"] * 1000 + path_df["GROUP_ID"] *
    10 + path_df["INDIVIDUAL_ID"]
)
ind_df["COMPOUND_NAME"] = ind_df["COMPOUND_NAME"].where(
    ind_df["COMPOUND_NAME"] != "TNFﾎｱ", "TNFα"
)

In [5]:
finding_lst = list(set(path_df["FIND_TYPE"].dropna()))
print(len(finding_lst))
path_df["FIND_TYPE"].head()

65


0      Deposit, glycogen
1      Deposit, glycogen
2    Degeneration, fatty
3    Degeneration, fatty
4    Degeneration, fatty
Name: FIND_TYPE, dtype: object

In [6]:
open(f"{PROCESSED_DIR}/finding_lst.txt", "w").write("\n".join(finding_lst) + "\n")

1199

In [7]:
one_hot = pd.get_dummies(
    path_df[["INDV_ID", "FIND_TYPE"]].dropna(),
    columns=["FIND_TYPE"],
    prefix="",
    prefix_sep="",
)
one_hot = one_hot.groupby("INDV_ID").max()
one_hot.head()

,"Accumulation, foam cell","Adenoma, hepatocellular","Alteration, cytoplasmic","Alteration, nuclear",Altered hepatocellular foci,Anisonucleosis,Atrophy,"Atypia, nuclear",Bacterium,Cellular foci,...,"Proliferation, Kupffer cell","Proliferation, bile duct","Proliferation, oval cell",Pyknosis,Scar,Single cell necrosis,Swelling,Thrombus,"Vacuolization, cytoplasmic","Vacuolization, nuclear"
INDV_ID,,,,,,,,,,,,,,,,,,,,,
40181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40192,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40193,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
ind_ft_df = pd.merge(ind_df, one_hot, on="INDV_ID", how="left").fillna(0)
ind_ft_df.head()

,EXP_ID,GROUP_ID,INDIVIDUAL_ID,INDV_ID,COMPOUND_NAME,COMPOUND_ABBR,COMPOUND_NO,SPECIES,TEST_TYPE,SIN_REP_TYPE,...,"Proliferation, Kupffer cell","Proliferation, bile duct","Proliferation, oval cell",Pyknosis,Scar,Single cell necrosis,Swelling,Thrombus,"Vacuolization, cytoplasmic","Vacuolization, nuclear"
0,40,1,1,40011,acetaminophen,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40,1,2,40012,acetaminophen,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40,1,3,40013,acetaminophen,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,40,1,4,40014,acetaminophen,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40,1,5,40015,acetaminophen,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
ind_ft_com_df = pd.merge(
    ind_ft_df, compound_df[["COMPOUND_NAME", "VEHICLE"]], on="COMPOUND_NAME", how="left"
)
ind_ft_com_df.head()

,EXP_ID,GROUP_ID,INDIVIDUAL_ID,INDV_ID,COMPOUND_NAME,COMPOUND_ABBR,COMPOUND_NO,SPECIES,TEST_TYPE,SIN_REP_TYPE,...,"Proliferation, bile duct","Proliferation, oval cell",Pyknosis,Scar,Single cell necrosis,Swelling,Thrombus,"Vacuolization, cytoplasmic","Vacuolization, nuclear",VEHICLE
0,40,1,1,40011,acetaminophen,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5% MC
1,40,1,2,40012,acetaminophen,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5% MC
2,40,1,3,40013,acetaminophen,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5% MC
3,40,1,4,40014,acetaminophen,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5% MC
4,40,1,5,40015,acetaminophen,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5% MC


In [10]:
ind_ft_com_df["COMPOUND_NAME"] = ind_ft_com_df["COMPOUND_NAME"].where(
    ind_ft_com_df["DOSE"] != 0, ind_ft_com_df["VEHICLE"]
)
ind_ft_com_df[ind_ft_com_df["DOSE"] == 0].head()

,EXP_ID,GROUP_ID,INDIVIDUAL_ID,INDV_ID,COMPOUND_NAME,COMPOUND_ABBR,COMPOUND_NO,SPECIES,TEST_TYPE,SIN_REP_TYPE,...,"Proliferation, bile duct","Proliferation, oval cell",Pyknosis,Scar,Single cell necrosis,Swelling,Thrombus,"Vacuolization, cytoplasmic","Vacuolization, nuclear",VEHICLE
0,40,1,1,40011,0.5% MC,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5% MC
1,40,1,2,40012,0.5% MC,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5% MC
2,40,1,3,40013,0.5% MC,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5% MC
3,40,1,4,40014,0.5% MC,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5% MC
4,40,1,5,40015,0.5% MC,APAP,1,Rat,in vivo,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5% MC


In [11]:
compound_list = list(set(ind_ft_com_df["COMPOUND_NAME"]))
print(*compound_list, sep="\t")

hexachlorobenzene	nicotinic acid	benzbromarone	terbinafine	flutamide	bromoethylamine	papaverine	fluoxetine hydrochloride	N-nitrosomorpholine	aflatoxin B1	moxisylyte	chlorpromazine	phorone	TNFα	diethyl maleate	clomipramine	bortezomib	nitrofurazone	ciprofloxacin	LPS	disulfiram	ethionamide	distilled water	perhexiline	2-nitrofluorene	fluphenazine	1% cholesterol + 0.25% sodium cholate	cisplatin	glibenclamide	ethambutol	quinidine	thioacetamide	tunicamycin	nitrosodiethylamine	amphotericin B	amiodarone	iproniazid	promethazine	meloxicam	phenacetin	allopurinol	triamterene	N-methyl-N-nitrosourea	griseofulvin	gentamicin	cimetidine	sulfasalazine	lornoxicam	bromobenzene	puromycin aminonucleoside	butylated hydroxyanisole	chloramphenicol	etoposide	coumarin	aspirin	ketoconazole	triazolam	nimesulide	labetalol	bendazac	allyl alcohol	phenylbutazone	methimazole	3-methylcholanthrene	trimethadione	erythromycin ethylsuccinate	tolbutamide	methyldopa	naproxen	acetamidofluorene	disopyramide	fenofibrate	simvastat

In [12]:
# saving the compound list
open(f"{PROCESSED_DIR}/compound_list.txt","w").write("\n".join(compound_list) + "\n")

2222

In [13]:
one_hot = pd.get_dummies(
    ind_ft_com_df[["INDV_ID", "COMPOUND_NAME"]], prefix="", prefix_sep=""
)
ind_ft_com_df = pd.merge(ind_ft_df, one_hot, on="INDV_ID", how="left")
ind_ft_com_df.head()

,EXP_ID,GROUP_ID,INDIVIDUAL_ID,INDV_ID,COMPOUND_NAME,COMPOUND_ABBR,COMPOUND_NO,SPECIES,TEST_TYPE,SIN_REP_TYPE,...,thioridazine,ticlopidine,tiopronin,tolbutamide,triamterene,triazolam,trimethadione,tunicamycin,valproic acid,vitamin A
0,40,1,1,40011,acetaminophen,APAP,1,Rat,in vivo,Single,...,0,0,0,0,0,0,0,0,0,0
1,40,1,2,40012,acetaminophen,APAP,1,Rat,in vivo,Single,...,0,0,0,0,0,0,0,0,0,0
2,40,1,3,40013,acetaminophen,APAP,1,Rat,in vivo,Single,...,0,0,0,0,0,0,0,0,0,0
3,40,1,4,40014,acetaminophen,APAP,1,Rat,in vivo,Single,...,0,0,0,0,0,0,0,0,0,0
4,40,1,5,40015,acetaminophen,APAP,1,Rat,in vivo,Single,...,0,0,0,0,0,0,0,0,0,0


In [14]:
image_df["FILE"] = [f[-1] for f in image_df["FILE_LOCATION"].str.split("/")]
image_df["FILE"].head()

0    26761.svs
2    26765.svs
4    26770.svs
6    26774.svs
8    26778.svs
Name: FILE, dtype: object

In [15]:
info_df = pd.merge(
    ind_ft_com_df,
    image_df[["INDV_ID", "FILE", "FILE_LOCATION"]],
    on="INDV_ID",
    how="left",
)
info_df.head()

,EXP_ID,GROUP_ID,INDIVIDUAL_ID,INDV_ID,COMPOUND_NAME,COMPOUND_ABBR,COMPOUND_NO,SPECIES,TEST_TYPE,SIN_REP_TYPE,...,tiopronin,tolbutamide,triamterene,triazolam,trimethadione,tunicamycin,valproic acid,vitamin A,FILE,FILE_LOCATION
0,40,1,1,40011,acetaminophen,APAP,1,Rat,in vivo,Single,...,0,0,0,0,0,0,0,0,26761.svs,ftp://ftp.biosciencedbc.jp/archive/open-tggate...
1,40,1,2,40012,acetaminophen,APAP,1,Rat,in vivo,Single,...,0,0,0,0,0,0,0,0,26765.svs,ftp://ftp.biosciencedbc.jp/archive/open-tggate...
2,40,1,3,40013,acetaminophen,APAP,1,Rat,in vivo,Single,...,0,0,0,0,0,0,0,0,26770.svs,ftp://ftp.biosciencedbc.jp/archive/open-tggate...
3,40,1,4,40014,acetaminophen,APAP,1,Rat,in vivo,Single,...,0,0,0,0,0,0,0,0,26774.svs,ftp://ftp.biosciencedbc.jp/archive/open-tggate...
4,40,1,5,40015,acetaminophen,APAP,1,Rat,in vivo,Single,...,0,0,0,0,0,0,0,0,26778.svs,ftp://ftp.biosciencedbc.jp/archive/open-tggate...


In [16]:
info_df.to_csv(f"{PROCESSED_DIR}/info.csv", index=False)